In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# machine learning modules
import sklearn
print(sklearn.__version__)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import metrics
from sklearn.metrics import roc_auc_score

0.19.0


In [3]:
train = pd.read_csv(".../train.csv",
                         na_values='-1')
train.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2.0,5,1.0,0.0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1.0,7,0.0,0.0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4.0,9,1.0,0.0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1.0,2,0.0,0.0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2.0,0,1.0,0.0,1,0,0,...,3,1,1,3,0,0,0,1,1,0


In [7]:
train_data_nato_median = pd.DataFrame()
for column in train.columns:
    train_data_nato_median[column] = train[column].fillna(train[column].median())

train_data = train_data_nato_median.copy()

In [8]:
test_data = pd.read_csv(".../test.csv", na_values='-1')
## Filling the missing data NAN with mean of the column
test_data_nato_median = pd.DataFrame()
for column in test_data.columns:
    test_data_nato_median[column] = test_data[column].fillna(test_data[column].median())
    
test_data = test_data_nato_median.copy()
test_data_id = test_data.pop('id')

In [9]:
column_names = train_data.columns
categorical_column = column_names[column_names.str[10] == 'c']
print categorical_column
print column_names
## Changing categorical columns to category data type
def int_to_categorical(data):
    """ 
    changing columns to catgorical data type
    """
    for column in categorical_column:
        data[column] =  data[column].astype('category')

Index([u'ps_ind_02_cat', u'ps_ind_04_cat', u'ps_ind_05_cat', u'ps_car_01_cat',
       u'ps_car_02_cat', u'ps_car_03_cat', u'ps_car_04_cat', u'ps_car_05_cat',
       u'ps_car_06_cat', u'ps_car_07_cat', u'ps_car_08_cat', u'ps_car_09_cat',
       u'ps_car_10_cat', u'ps_car_11_cat'],
      dtype='object')
Index([u'id', u'target', u'ps_ind_01', u'ps_ind_02_cat', u'ps_ind_03',
       u'ps_ind_04_cat', u'ps_ind_05_cat', u'ps_ind_06_bin', u'ps_ind_07_bin',
       u'ps_ind_08_bin', u'ps_ind_09_bin', u'ps_ind_10_bin', u'ps_ind_11_bin',
       u'ps_ind_12_bin', u'ps_ind_13_bin', u'ps_ind_14', u'ps_ind_15',
       u'ps_ind_16_bin', u'ps_ind_17_bin', u'ps_ind_18_bin', u'ps_reg_01',
       u'ps_reg_02', u'ps_reg_03', u'ps_car_01_cat', u'ps_car_02_cat',
       u'ps_car_03_cat', u'ps_car_04_cat', u'ps_car_05_cat', u'ps_car_06_cat',
       u'ps_car_07_cat', u'ps_car_08_cat', u'ps_car_09_cat', u'ps_car_10_cat',
       u'ps_car_11_cat', u'ps_car_11', u'ps_car_12', u'ps_car_13',
       u'ps_car_14', u'ps_

In [10]:
datas = [train_data,test_data]

for data in datas:
    int_to_categorical(data)

print(test_data.dtypes)

ps_ind_01            int64
ps_ind_02_cat     category
ps_ind_03            int64
ps_ind_04_cat     category
ps_ind_05_cat     category
ps_ind_06_bin        int64
ps_ind_07_bin        int64
ps_ind_08_bin        int64
ps_ind_09_bin        int64
ps_ind_10_bin        int64
ps_ind_11_bin        int64
ps_ind_12_bin        int64
ps_ind_13_bin        int64
ps_ind_14            int64
ps_ind_15            int64
ps_ind_16_bin        int64
ps_ind_17_bin        int64
ps_ind_18_bin        int64
ps_reg_01          float64
ps_reg_02          float64
ps_reg_03          float64
ps_car_01_cat     category
ps_car_02_cat     category
ps_car_03_cat     category
ps_car_04_cat     category
ps_car_05_cat     category
ps_car_06_cat     category
ps_car_07_cat     category
ps_car_08_cat     category
ps_car_09_cat     category
ps_car_10_cat     category
ps_car_11_cat     category
ps_car_11          float64
ps_car_12          float64
ps_car_13          float64
ps_car_14          float64
ps_car_15          float64
p

In [11]:
def creating_dummies(data):
    """creating dummies columns categorical varibles
    """
    for column in categorical_column:
        dummies = pd.get_dummies(data[column],prefix=column)
        data = pd.concat([data,dummies],axis =1)
        ## dropping the original columns ##
        data.drop([column],axis=1,inplace= True)

In [12]:
for column in categorical_column:
    dummies = pd.get_dummies(train_data[column],prefix=column)
    train_data = pd.concat([train_data,dummies],axis =1)
    train_data.drop([column],axis=1,inplace= True)


for column in categorical_column:
        dummies = pd.get_dummies(test_data[column],prefix=column)
        test_data = pd.concat([test_data,dummies],axis =1)
        test_data.drop([column],axis=1,inplace= True)

print(train_data.shape)
print(test_data.shape)

(595212, 220)
(892816, 218)


In [13]:
#Define covariates in X and dependent variable in y
X = train_data.iloc[:,2:] ## FEATURE DATA
y= train_data.target ### LABEL DATA

### CHECKING DIMENSIONS
print(X.shape)
print(y.shape)

(595212, 218)
(595212,)


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)


### RANDOM FOREST CLASSIFIER

"""
number of estimators: 200
out of bagging set to True
N_jobs: Use all the available cores= -1
min_sample_leaf: minimum number of samples required to be at a leaf node
"""
RF_model_cat= RandomForestClassifier(200,oob_score=True,random_state=13,
                                     n_jobs = -1, min_samples_leaf = 100)

In [15]:
RF_model_cat.fit(X_train, y_train)

#Obtain class predictions
y_pred_RF_prob = RF_model_cat.predict_proba(X_test)
print('Predicted probabilities: \n', y_pred_RF_prob)

#Obtain probability predictions
y_pred_RF_class = RF_model_cat.predict(X_test)
print('Predicted classes: \n', y_pred_RF_class)

print('RF Score: ', metrics.accuracy_score(y_test, y_pred_RF_class))

## CONFUSION MATRIX
RF_cm=metrics.confusion_matrix(y_test,y_pred_RF_class)
print(RF_cm)

('Predicted probabilities: \n', array([[ 0.97157923,  0.02842077],
       [ 0.96437336,  0.03562664],
       [ 0.97800145,  0.02199855],
       ..., 
       [ 0.97366796,  0.02633204],
       [ 0.97002314,  0.02997686],
       [ 0.96032785,  0.03467215]]))
('Predicted classes: \n', array([0, 0, 0, ..., 0, 0, 0]))
('RF Score: ', 0.96337439433344763)
[[143353      0]
 [  5450      0]]


In [17]:
y_pred_RF_prob = RF_model_cat.predict_proba(test_data)
pred_values= pd.DataFrame(y_pred_RF_prob)

submission_simple_RF= pd.DataFrame()
submission_simple_RF['id'] = test_data_id

submission_simple_RF['target'] = pd.DataFrame(pred_values.iloc[:,1])
submission_simple_RF = submission_simple_RF.set_index('id')

submission_simple_RF.columns
submission_simple_RF.head()
## Write to CSV
submission_simple_RF.to_csv("Simple Random Forest.csv")